In [5]:
import string
import pandas as pd
import operator
from os import listdir
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
files = listdir(r"C:\Users\gamb desktop\Downloads\Compressed\20_newsgroups\20_newsgroups")
dictionary={}
for i in files:
    for j in listdir(r"C:\Users\gamb desktop\Downloads\Compressed\20_newsgroups\20_newsgroups\\" + i):
        current_file = open(r"C:\Users\gamb desktop\Downloads\Compressed\20_newsgroups\20_newsgroups\\" + i +"\\" + j,'r')
        text = current_file.read().replace('\n',' ').split()
        for word in text:
            if (word.lower() in stop) or (word in string.punctuation):
                continue
            else:
                if word in dictionary:
                    dictionary[word]=dictionary[word]+1
                else:
                    dictionary[word]=1

In [2]:
import numpy as np
sorted_y=sorted(dictionary.items(),key=operator.itemgetter(1),reverse=True)
new_d={}
for i,j in sorted_y:
    new_d[i]=j
new_dict={}
for i,j in sorted_y:
    new_dict[i]=j
l=[]
for i in new_dict:
    l.append(new_dict[i])
m=np.array(l)
n=m[:2000]

In [3]:
final_dictionary={}
counts=0
for i in new_dict:
    if counts == 2000:
        break
    final_dictionary[i]=counts #basically now i am storing the position present in dictionary as value(easy in x_train)
    counts+=1

In [4]:
import pandas as pd
new_file=listdir(r"C:\Users\gamb desktop\Downloads\Compressed\20_newsgroups\20_newsgroups")
counts=0
x_total=[]
y_total=[]
for i in new_file:
    for j in listdir(r"C:\Users\gamb desktop\Downloads\Compressed\20_newsgroups\20_newsgroups\\"+i):
        current_new_file=open(r"C:\Users\gamb desktop\Downloads\Compressed\20_newsgroups\20_newsgroups\\"+i+"\\"+j,'r')
        new_text=current_new_file.read().replace('\n',' ').split()
        temp=[]
        for alpha in range(2000):
            temp.append(0)
        for k in new_text:
            if k in final_dictionary:
                temp[final_dictionary[k]]+=1
        x_total.append(temp)
        y_total.append(counts)
    counts+=1

In [5]:
def train(x_train,y_train,final_dictionary):
    real_dictionary={}
    y_set=set(y_train)
    real_dictionary["total_length_of_y"]=len(y_train)
    for classes in y_set:
        real_dictionary[classes]={}
        x_train_classes=x_train[y_train==classes]
        y_train_classes=y_train[y_train==classes]
        real_dictionary[classes]["total_counts_of_class"]=len(y_train_classes)
        total_counts=0
        for word in final_dictionary.keys():
            counts=x_train_classes[:,final_dictionary[word]].sum()
            total_counts+=counts
            real_dictionary[classes][final_dictionary[word]]=counts
        real_dictionary[classes]["total__counts_of_words"]=total_counts
    return real_dictionary

In [6]:
def single_point_prediction(x,final_dictionary,real_dictionary):
    maxi_prob=-10000000
    class_at_maxi_prob=-1
    for classes in real_dictionary.keys():
        if classes=="total_length_of_y":
            continue
        prob=np.log(real_dictionary[classes]["total_counts_of_class"])-np.log(real_dictionary["total_length_of_y"])
        counts=0
        for i in x:
            if i>0:
                left=np.log(i*(real_dictionary[classes][counts]+1))
                right=np.log(i*(real_dictionary[classes]["total__counts_of_words"]+len(final_dictionary.keys())))
                prob=prob+left-right
            counts+=1
        if prob>maxi_prob:
            maxi_prob=prob
            class_at_maxi_prob=classes
    return class_at_maxi_prob

In [7]:
def predict(x_test,final_dictionary,real_dictionary):
    y_pred=[]
    for x in x_test:
        #print(x,x.sum())
        predicted_class=single_point_prediction(x,final_dictionary,real_dictionary)
        print(predicted_class)
        y_pred.append(predicted_class)
    return y_pred

In [8]:
from sklearn import model_selection
from collections import Counter
import numpy as np
import pandas as pd
x_train,x_test,y_train,y_test=model_selection.train_test_split(x_total,y_total,test_size=0.2,random_state=0)
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [9]:
real_dictionary=train(x_train,y_train,final_dictionary)

In [10]:
y_pred=predict(x_test,final_dictionary,real_dictionary)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
#print(y_pred)

5
13
16
6
10
19
3
2
7
1
16
4
16
15
4
3
3
12
2
19
16
6
19
9
12
4
4
11
13
11
14
6
15
13
15
4
19
13
9
8
15
2
17
14
4
17
13
16
0
3
16
2
18
19
17
9
10
3
18
15
1
10
2
8
17
10
4
0
0
9
4
6
10
1
11
14
4
11
2
6
2
6
4
15
3
15
15
6
8
3
7
4
7
16
17
19
13
6
13
8
18
8
2
9
17
8
2
6
4
19
5
9
6
5
13
4
1
8
19
17
6
13
1
8
4
19
6
7
8
19
3
11
6
7
2
13
10
8
19
8
16
7
7
12
6
10
17
5
11
4
15
14
9
13
6
3
14
1
15
17
2
5
12
2
6
19
7
7
15
0
4
15
14
10
6
8
12
14
7
7
2
0
7
1
0
18
14
7
15
4
6
3
13
14
0
15
15
2
10
16
16
15
10
8
14
10
2
6
10
16
6
17
3
10
3
9
9
4
11
13
15
12
0
13
0
6
13
9
15
1
13
16
5
4
4
13
8
4
2
14
12
2
18
3
1
0
3
2
13
3
15
3
3
19
14
13
2
5
4
4
9
17
14
7
14
19
15
16
3
0
10
14
15
2
13
9
1
12
13
16
12
0
10
12
16
12
6
13
5
0
13
7
4
17
11
16
14
16
4
0
7
10
11
15
13
12
15
12
7
3
13
3
7
14
13
6
7
6
14
7
3
14
6
11
5
16
17
6
2
9
5
12
13
13
8
13
1
6
19
3
3
14
6
18
2
12
11
17
9
12
7
1
12
0
13
11
0
6
14
6
2
8
12
14
7
4
3
2
18
11
6
4
9
1
19
11
1
6
9
7
13
1
1
15
18
12
6
17
10
10
1
0
14
1
6
9
2
5
6
14
4
3
0
2
11
0
